In [2]:
import numpy as np
import pandas as pd
from pandas.tseries.offsets import DateOffset
from prophet import Prophet
#from neuralprophet import NeuralProphet
import openpyxl as op
from tqdm.auto import tqdm
import plotly.graph_objs as go
import time
import plotly.express as px
import warnings
warnings.filterwarnings("ignore", message="No frequency information was")

import logging
logger = logging.getLogger('cmdstanpy')
logger.addHandler(logging.NullHandler())
logger.propagate = False
logger.setLevel(logging.CRITICAL)

instances = pd.read_csv("../instances/instances.csv", index_col=0).to_numpy().reshape(14)

In [3]:
INSTANCE = 0

In [4]:
data_series = pd.read_csv("../instances/{n}/series_{n}.csv".format(n=instances[INSTANCE]))
data_series['local_15min'] = pd.to_datetime(data_series['local_15min'])
data_series.index = data_series['local_15min']
data_series.drop(columns=['local_15min'], inplace=True)
data_pv = data_series[['solar']]

In [35]:
start_date = "2019-05-01"
end_date = "2019-08-01"
training_data = data_pv[(data_pv.index >= start_date) & (data_pv.index<end_date)]

In [25]:
def prophet_ses_model(training_data, alpha, d, timesteps, days, plot=False, path_plot=None):
    
    k = timesteps*days

    # Inicializar df con la información
    df = pd.DataFrame()

    # Calcular cantidad de días a pronósticar
    days_forecast = training_data.index[-1]-training_data.index[k-1]
    days_forecast = days_forecast.days

    for i in range(days_forecast):
        
        # Training set
        train = training_data[96*i:k+96*i]
        model = Prophet(growth='flat')
        model.fit(train.reset_index().rename(columns={"local_15min": "ds", "solar": "y"}))
        #model = sm.tsa.SARIMAX(train['solar'].asfreq("15min"), order = order,seasonal_order=seasonal_order, freq=train.index.inferred_freq).fit()

        future = model.make_future_dataframe(periods=96, freq="15min", include_history=False)
        forecast = model.predict(future)

        forecast['yhat'] = forecast['yhat'].apply(lambda x: 0 if x < 0 else x)   # Replace negative predictions with zero
        forecast['hour'] = forecast['ds'].dt.hour
        forecast.loc[((forecast['hour']>=21)|(forecast['hour']<=4)), 'yhat'] = 0 # Replace predicions between 8pm and 4am with zero
        
        forecast = forecast[['ds', 'yhat']].merge(training_data, how='left', left_on='ds', right_index=True)
        forecast['e'] = forecast['solar'] - forecast['yhat']

        ses = []

        for j in range(96):
   
            df_forecast = forecast[j:].copy().reset_index(drop=True)
            forecast_dates = df_forecast['ds']
            
            # Calcular nuevo valor del error
            if j==0:
                error = 0
            elif j==1:
                error = forecast.loc[j-1, 'e']
                ses.append(error)
            else:
                error = ses[-1] + alpha*(forecast.loc[j-1, 'e']-ses[-1])
                ses.append(error)
            
            decay = np.array([d**x for x in range(len(forecast_dates))])
            df_forecast['e_pred'] = error*decay 

            # Update dataframe
            df_forecast['From'] = train.index[-1] + DateOffset(minutes=15*j)
            df_forecast['Day'] = df_forecast['ds'].dt.date
            df = pd.concat([df, df_forecast.reset_index()])
        
        if plot==True:
            plot_model(df_forecast, path_plot, i)

        #print(str(i)+"|"+str(days_forecast-1),'train:', train.index[0], '->', train.index[-1], '|forecast:', forecast_dates[0], '->', forecast_dates[-1], '|mse:', df_forecast['MSE'].mean())

    df['temp'] = -1*df['yhat']
    df['e_pred'] = df[['e_pred', 'temp']].max(axis=1)

    df.drop(columns=['index'], inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    df['hour'] = df['ds'].dt.hour
    df.loc[((df['hour']>=21)|( df['hour']<=4)), 'e_pred'] = df['temp']

    return df

In [19]:
def auto_prophet_ses(training_data, alpha_limits, decay_limits, ruta, timesteps, days):

    # Crear libro de excel
    book = op.Workbook()
    sheet = book.active
    sheet.title = 'Resultados'
    sheet.cell(3,1).value = 'alpha'
    sheet.cell(3,2).value = 'decay'
    sheet.cell(3,3).value = 'mse_mean'
    sheet.cell(3,4).value = 'mse_sd'

    # Inicializar lista con el mse de cada modelo y el contador
    mse_models = {}
    c = 3
    for a in alpha_limits:   
        for d in decay_limits:

            df = prophet_ses_model(training_data=training_data, alpha=a, d=d, timesteps=timesteps, days=days)
            
            df['mse'] = (df['e']-df['e_pred'])**2
            mse_mean = df['mse'].mean()
            print(" Model:", "({}, {}) --> MSE: {}".format(a, d, np.round(mse_mean,4)))
            mse_sd = df['mse'].var()**(1/2)
            mse_models["({}, {})".format(a,d)] = mse_mean
            c = c+1
            sheet.cell(c,1).value = a
            sheet.cell(c,2).value = d
            sheet.cell(c,3).value = mse_mean
            sheet.cell(c,4).value = mse_sd

            book.save(ruta)

    # best model
    sheet.cell(1,1).value = 'Mejor modelo:'
    sheet.cell(1,2).value = min(mse_models, key=mse_models.get)
    sheet.cell(1,3).value = min(mse_models.values())

    book.save(ruta)

In [25]:
# Parámetros
timesteps = 96
days = 15 # 5 dias de observación
ruta = 'resultados_fixed/15min/resultados_prophet_ses.xlsx'

alpha_limits = [0.1, 0.3, 0.5, 0.7, 0.9]
decay_limits = [0.3, 0.6, 0.8, 0.9, 0.99, 1]

#auto_prophet_ses(training_data=training_data, alpha_limits=alpha_limits, decay_limits=decay_limits, ruta=ruta, timesteps=timesteps, days=days)
    

  0%|          | 0/61 [00:00<?, ?it/s]

 Model: (0.1, 0.3)


  0%|          | 0/61 [00:00<?, ?it/s]

0.0234
 Model: (0.1, 0.6)


  0%|          | 0/61 [00:00<?, ?it/s]

0.0231
 Model: (0.1, 0.8)


  0%|          | 0/61 [00:00<?, ?it/s]

0.0226
 Model: (0.1, 0.9)


 18%|█▊        | 11/61 [00:03<00:13,  3.64it/s]


KeyboardInterrupt: 

In [48]:
for i in tqdm(range(len(instances))):

    if i==0:
        continue
    data_series = pd.read_csv("../instances/{n}/series_{n}.csv".format(n=instances[i]))
    data_series['local_15min'] = pd.to_datetime(data_series['local_15min'])
    data_series.index = data_series['local_15min']
    data_series.drop(columns=['local_15min'], inplace=True)
    data_pv = data_series[['solar']]

    #start_date = "2019-05-17"
    start_date = "2019-05-01"
    end_date = "2019-08-01"
    training_data = data_pv[(data_pv.index >= start_date) & (data_pv.index<end_date)]

    timesteps = 96
    days = 31 # 15 dias de observación
    ruta = 'resultados/15min/prophet_ses/prophet_ses_31days_i{}.xlsx'.format(i)
    alpha_limits = [0.3, 0.5, 0.7, 0.9]
    #alpha_limits = [0.9]
    decay_limits = [0.6, 0.8, 0.9, 0.99, 1]
    #decay_limits = [0.9]
    auto_prophet_ses(training_data=training_data, alpha_limits=alpha_limits, decay_limits=decay_limits, ruta=ruta, timesteps=timesteps, days=days)


  0%|          | 0/14 [00:00<?, ?it/s]

 Model: (0.3, 0.6) --> MSE: 0.0206
 Model: (0.3, 0.8) --> MSE: 0.0199
 Model: (0.3, 0.9) --> MSE: 0.0191
 Model: (0.3, 0.99) --> MSE: 0.0196
 Model: (0.3, 1) --> MSE: 0.0204
 Model: (0.5, 0.6) --> MSE: 0.0205
 Model: (0.5, 0.8) --> MSE: 0.0198
 Model: (0.5, 0.9) --> MSE: 0.019
 Model: (0.5, 0.99) --> MSE: 0.0197
 Model: (0.5, 1) --> MSE: 0.0207
 Model: (0.7, 0.6) --> MSE: 0.0205
 Model: (0.7, 0.8) --> MSE: 0.0197
 Model: (0.7, 0.9) --> MSE: 0.0189
 Model: (0.7, 0.99) --> MSE: 0.0198
 Model: (0.7, 1) --> MSE: 0.0209
 Model: (0.9, 0.6) --> MSE: 0.0205
 Model: (0.9, 0.8) --> MSE: 0.0197
 Model: (0.9, 0.9) --> MSE: 0.0189
 Model: (0.9, 0.99) --> MSE: 0.02


 14%|█▍        | 2/14 [10:18<1:01:53, 309.47s/it]

 Model: (0.9, 1) --> MSE: 0.0212
 Model: (0.3, 0.6) --> MSE: 0.1366
 Model: (0.3, 0.8) --> MSE: 0.1317
 Model: (0.3, 0.9) --> MSE: 0.1261
 Model: (0.3, 0.99) --> MSE: 0.1341
 Model: (0.3, 1) --> MSE: 0.1462
 Model: (0.5, 0.6) --> MSE: 0.1362
 Model: (0.5, 0.8) --> MSE: 0.1309
 Model: (0.5, 0.9) --> MSE: 0.1251
 Model: (0.5, 0.99) --> MSE: 0.1356
 Model: (0.5, 1) --> MSE: 0.15
 Model: (0.7, 0.6) --> MSE: 0.136
 Model: (0.7, 0.8) --> MSE: 0.1307
 Model: (0.7, 0.9) --> MSE: 0.1248
 Model: (0.7, 0.99) --> MSE: 0.1369
 Model: (0.7, 1) --> MSE: 0.1526
 Model: (0.9, 0.6) --> MSE: 0.1359
 Model: (0.9, 0.8) --> MSE: 0.1306
 Model: (0.9, 0.9) --> MSE: 0.1248
 Model: (0.9, 0.99) --> MSE: 0.1383


 21%|██▏       | 3/14 [8:23:28<38:13:23, 12509.41s/it]

...

 Model: (0.9, 1) --> MSE: 0.0358
 Model: (0.3, 0.6) --> MSE: 0.1065
 Model: (0.3, 0.8) --> MSE: 0.1034
 Model: (0.3, 0.9) --> MSE: 0.1009
 Model: (0.3, 0.99) --> MSE: 0.1086
 Model: (0.3, 1) --> MSE: 0.1141
 Model: (0.5, 0.6) --> MSE: 0.1062
 Model: (0.5, 0.8) --> MSE: 0.1028
 Model: (0.5, 0.9) --> MSE: 0.1001
 Model: (0.5, 0.99) --> MSE: 0.1095
 Model: (0.5, 1) --> MSE: 0.1162
 Model: (0.7, 0.6) --> MSE: 0.1061
 Model: (0.7, 0.8) --> MSE: 0.1026
 Model: (0.7, 0.9) --> MSE: 0.0999
 Model: (0.7, 0.99) --> MSE: 0.1105
 Model: (0.7, 1) --> MSE: 0.118
 Model: (0.9, 0.6) --> MSE: 0.1061
 Model: (0.9, 0.8) --> MSE: 0.1026
 Model: (0.9, 0.9) --> MSE: 0.0999
 Model: (0.9, 0.99) --> MSE: 0.1119


 93%|█████████▎| 13/14 [12:01:28<16:07, 967.75s/it] 

 Model: (0.9, 1) --> MSE: 0.1199
 Model: (0.3, 0.6) --> MSE: 0.1279
 Model: (0.3, 0.8) --> MSE: 0.1234
 Model: (0.3, 0.9) --> MSE: 0.1195
 Model: (0.3, 0.99) --> MSE: 0.1317
 Model: (0.3, 1) --> MSE: 0.1423
 Model: (0.5, 0.6) --> MSE: 0.1275
 Model: (0.5, 0.8) --> MSE: 0.1227
 Model: (0.5, 0.9) --> MSE: 0.1184
 Model: (0.5, 0.99) --> MSE: 0.133
 Model: (0.5, 1) --> MSE: 0.1455
 Model: (0.7, 0.6) --> MSE: 0.1273
 Model: (0.7, 0.8) --> MSE: 0.1225
 Model: (0.7, 0.9) --> MSE: 0.1181
 Model: (0.7, 0.99) --> MSE: 0.1344
 Model: (0.7, 1) --> MSE: 0.1481
 Model: (0.9, 0.6) --> MSE: 0.1273
 Model: (0.9, 0.8) --> MSE: 0.1224
 Model: (0.9, 0.9) --> MSE: 0.1182
 Model: (0.9, 0.99) --> MSE: 0.1362


100%|██████████| 14/14 [12:11:02<00:00, 3133.02s/it]

 Model: (0.9, 1) --> MSE: 0.1509


In [36]:
# Mejor modelo

# Parámetros
timesteps = 96
df = prophet_ses_model(training_data=training_data, alpha=0.9, d=0.9, days=31, timesteps=timesteps)

df['mse'] = (df['e']-df['e_pred'])**2
df['mae'] = abs(df['e']-df['e_pred'])

mse_mean = df['mse'].mean()
mse_sd = df['mse'].var()**(1/2)
mae = df['mae'].mean()

print('mse:', mse_mean)
print('mse_sd:',mse_sd)
print('mae',np.round(mae,2))


mse: 0.019636963997391017
mse_sd: 0.052597447277300556
mae 0.08


In [37]:
df['y_pred'] = df['yhat']+df['e_pred']
dias = df['Day'].unique()

df_plot = df.copy()
df_plot['From'] = df_plot['From'].astype(str)
df_plot.head(30)

,ds,yhat,solar,e,e_pred,From,Day,temp,hour,mse,mae,y_pred
0,2019-06-01 00:00:00,0.000000,0.00175,0.001750,-0.0,2019-05-31 23:45:00,2019-06-01,-0.000000,0,0.000003,0.001750,0.000000
1,2019-06-01 00:15:00,0.000000,0.00175,0.001750,-0.0,2019-05-31 23:45:00,2019-06-01,-0.000000,0,0.000003,0.001750,0.000000
2,2019-06-01 00:30:00,0.000000,0.00175,0.001750,-0.0,2019-05-31 23:45:00,2019-06-01,-0.000000,0,0.000003,0.001750,0.000000
3,2019-06-01 00:45:00,0.000000,0.00175,0.001750,-0.0,2019-05-31 23:45:00,2019-06-01,-0.000000,0,0.000003,0.001750,0.000000
4,2019-06-01 01:00:00,0.000000,0.00175,0.001750,-0.0,2019-05-31 23:45:00,2019-06-01,-0.000000,1,0.000003,0.001750,0.000000
5,2019-06-01 01:15:00,0.000000,0.00175,0.001750,-0.0,2019-05-31 23:45:00,2019-06-01,-0.000000,1,0.000003,0.001750,0.000000
6,2019-06-01 01:30:00,0.000000,0.00175,0.001750,-0.0,2019-05-31 23:45:00,2019-06-01,-0.000000,1,0.000003,0.001750,0.000000
7,2019-06-01 01:45:00,0.000000,0.00175,0.001750,-0.0,2019-05-31 23:45:00,2019-06-01,-0.000000,1,0.000003,0.001750,0.000000
8,2019-06-01 02:00:00,0.000000,0.00200,0.002000,-0.0,2019-05-31 23:45:00,2019-06-01,-0.000000,2,0.000004,0.002000,0.000000
9,2019-06-01 02:15:00,0.000000,0.00200,0.002000,-0.0,2019-05-31 23:45:00,2019-06-01,-0.000000,2,0.000004,0.002000,0.000000


In [38]:
df_plot[df_plot.From=='2019-06-01 00:15:00'].reset_index(drop=True).tail(30)

,ds,yhat,solar,e,e_pred,From,Day,temp,hour,mse,mae,y_pred
64,2019-06-01 16:30:00,0.137240,0.17800,0.040760,2.063282e-06,2019-06-01 00:15:00,2019-06-01,-0.137240,16,1.661240e-03,0.040758,1.372417e-01
65,2019-06-01 16:45:00,0.114900,0.14650,0.031600,1.856954e-06,2019-06-01 00:15:00,2019-06-01,-0.114900,16,9.984267e-04,0.031598,1.149021e-01
66,2019-06-01 17:00:00,0.095660,0.12175,0.026090,1.671259e-06,2019-06-01 00:15:00,2019-06-01,-0.095660,17,6.805968e-04,0.026088,9.566175e-02
67,2019-06-01 17:15:00,0.079350,0.09775,0.018400,1.504133e-06,2019-06-01 00:15:00,2019-06-01,-0.079350,17,3.385123e-04,0.018399,7.935130e-02
68,2019-06-01 17:30:00,0.065725,0.07625,0.010525,1.353720e-06,2019-06-01 00:15:00,2019-06-01,-0.065725,17,1.107438e-04,0.010523,6.572651e-02
69,2019-06-01 17:45:00,0.054486,0.06750,0.013014,1.218348e-06,2019-06-01 00:15:00,2019-06-01,-0.054486,17,1.693292e-04,0.013013,5.448734e-02
70,2019-06-01 18:00:00,0.045297,0.05075,0.005453,1.096513e-06,2019-06-01 00:15:00,2019-06-01,-0.045297,18,2.972283e-05,0.005452,4.529814e-02
71,2019-06-01 18:15:00,0.037807,0.02925,-0.008557,9.868615e-07,2019-06-01 00:15:00,2019-06-01,-0.037807,18,7.323604e-05,0.008558,3.780781e-02
72,2019-06-01 18:30:00,0.031668,0.01500,-0.016668,8.881754e-07,2019-06-01 00:15:00,2019-06-01,-0.031668,18,2.778545e-04,0.016669,3.166897e-02
73,2019-06-01 18:45:00,0.026554,0.00650,-0.020054,7.993578e-07,2019-06-01 00:15:00,2019-06-01,-0.026554,18,4.022026e-04,0.020055,2.655499e-02


In [39]:
df_real= df_plot[['ds', 'solar']].drop_duplicates()

In [40]:
df_plot["From2"] = "Forecast at " + df_plot["From"].str.slice(start=-8, stop=-3)

In [47]:
DIAS = dias[11:12]

colorscale = []
rgb = px.colors.convert_colors_to_same_type(px.colors.sequential.haline)[0]
n_steps = int(np.round(96/(len(rgb)-1)))  # Control the number of colors in the final colorscale
for i in range(len(rgb) - 1):
    for step in np.linspace(0, 1, n_steps):
        colorscale.append(px.colors.find_intermediate_color(rgb[i], rgb[i + 1], step, colortype='rgb'))

for i in range(len(colorscale)):
    colorscale[i] = colorscale[i][:-1]+', 0.5)'
    colorscale[i] = 'rgba'+colorscale[i][3:]

fig = px.line(df_plot[df_plot['Day'].isin(DIAS)], x="ds", y="y_pred", color='From2',color_discrete_sequence=colorscale, labels={"From2": ""})

labels_to_show_in_legend = ["Forecast at 0{}:00".format(i) for i in range(0, 10, 2)]+["Forecast at {}:00".format(i) for i in range(10, 24, 2)]

for trace in fig['data']: 
    if (not trace['name'] in labels_to_show_in_legend):
        trace['showlegend'] = False
fig.add_trace(go.Scatter(x=df_real[df_real['ds'].dt.date.isin(pd.to_datetime(DIAS).date)]["ds"], 
                         y=df_real[df_real['ds'].dt.date.isin(pd.to_datetime(DIAS).date)]["solar"], 
                         name="Actual PV generation",
                         line=dict(color='black'),))

fig.update_layout(title_text="Rolling forecast of PV energy",
                  yaxis=dict(title="Energy [kWh]"),
                  xaxis=dict(title= "Time"))
fig.write_image("../../Soportes documento/Graficas documento/pv_forecast.png", scale=2, height = 450, width=700)
fig.show()

In [146]:
df.shape

(284016, 13)